We’ll be working with a dataset from the bike-sharing service Hubway, which includes data on over 1.5 million trips made with the service.

### Our Data Set

Let’s start by taking a look at our database. The database has two tables, trips and stations. To begin with, we’ll just look at the trips table, which contains the following columns:

* id - A unique integer that serves as a reference for each trip
* duration - The duration of the trip, measured in seconds
* start_date - The date and time the trip began
* start_station - An integer that corresponds to the id column in the stations table for the station the trip started at
* end_date - The date and time the trip ended
* end_station - The ‘id’ of the station the trip ended at
* bike_number - Hubway’s unique identifier for the bike used on the trip
* sub_type - The subscription type of the user. "Registered" for users with a membership, "Casual" for users without a membership
* zip_code - The zip code of the user (only available for registered members)
* birth_date - The birth year of the user (only available for registered * members)
* gender - The gender of the user (only available for registered members)

### Our Analysis
With this information and the SQL commands we’ll learn shortly, here are some questions that we’ll try to answer over the course of this post

* What was the duration of the longest trip?
* How many trips were taken by ‘registered’ users?
* What was the average trip duration?
* Do registered or casual users take longer trips?
* Which bike was used for the most trips?
* What is the average duration of trips by users over the age of 30?

In [6]:
import sqlite3
import pandas as pd

db = sqlite3.connect('hubway.db')

def run_query(query):
    return pd.read_sql_query(query,db)

### What was the duration of the longest trip?

* Use SELECT to retrieve the duration column FROM the trips table
* Use ORDER BY to sort the duration column and use the DESC keyword to specify that you want to sort in descending order
* Use LIMIT to restrict the output to 1 row

In [24]:
## SELECT and LIMIT`

This query uses * as a wildcard instead of specifying columns to return. This means the SELECT command has given us every column in the trips table. We also used the LIMIT function to restrict the output to the first five rows of the table.

In [7]:
query = 'SELECT * FROM trips LIMIT 5;'
run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,1,9,2011-07-28 10:12:00,23,2011-07-28 10:12:00,23,B00468,Registered,'97217,1976.0,Male
1,2,220,2011-07-28 10:21:00,23,2011-07-28 10:25:00,23,B00554,Registered,'02215,1966.0,Male
2,3,56,2011-07-28 10:33:00,23,2011-07-28 10:34:00,23,B00456,Registered,'02108,1943.0,Male
3,4,64,2011-07-28 10:35:00,23,2011-07-28 10:36:00,23,B00554,Registered,'02116,1981.0,Female
4,5,12,2011-07-28 10:37:00,23,2011-07-28 10:37:00,23,B00554,Registered,'97214,1983.0,Female


The previous example returned every column in the trips table. If we were only interested in the duration and  start_date columns, we could replace the wildcard with the column names as follows:

In [8]:
query = 'SELECT duration, start_date FROM trips LIMIT 5'
run_query(query)

,duration,start_date
0,9,2011-07-28 10:12:00
1,220,2011-07-28 10:21:00
2,56,2011-07-28 10:33:00
3,64,2011-07-28 10:35:00
4,12,2011-07-28 10:37:00


In [10]:
query = '''
SELECT duration 
FROM trips
ORDER BY duration DESC
LIMIT 1;
'''

run_query(query)

,duration
0,9999


Now we know that the longest trip lasted 9999 seconds, or a little over 166 minutes. With a maximum value of 9999, however, we don’t know whether this is really the length of the longest trip or if the database was only set up to allow a four digit number. If it’s that particularly long trips are being cut short by the database, then we might expect to see a lot of trips at 9999 seconds where they reach the limit. Let’s try running the same query as before, but adjust the LIMIT to return the 10 highest durations and see if that’s the case:

In [11]:
query = '''
SELECT duration
FROM trips
ORDER BY duration DESC
LIMIT 10
'''
run_query(query)

,duration
0,9999
1,9998
2,9998
3,9997
4,9996
5,9996
6,9995
7,9995
8,9994
9,9994


What we see here is that there aren’t a whole bunch of trips at 9999, so it doesn’t look like we’re cutting off the top end of our durations, but it’s still difficult to tell whether that’s the real length of the trip or just the maximum allowed value. Hubway charges additional fees for rides over 30 minutes (somebody keeping a bike for 9999 seconds would have to pay an extra $25 in fees) so it’s plausible that they decided 4 digits would be sufficient to track the majority of rides.

### WHERE

Let’s write a query that uses WHERE to return every column in the trips table for each row with a duration longer than 9990 seconds:

In [12]:
query = '''
SELECT * 
FROM trips
WHERE duration > 9990;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,4768,9994,2011-08-03 17:16:00,22,2011-08-03 20:03:00,24,B00002,Casual,,,
1,8448,9991,2011-08-06 13:02:00,52,2011-08-06 15:48:00,24,B00174,Casual,,,
2,11341,9998,2011-08-09 10:42:00,40,2011-08-09 13:29:00,42,B00513,Casual,,,
3,24455,9995,2011-08-20 12:20:00,52,2011-08-20 15:07:00,17,B00552,Casual,,,
4,55771,9994,2011-09-14 15:44:00,40,2011-09-14 18:30:00,40,B00139,Casual,,,
5,81191,9993,2011-10-03 11:30:00,22,2011-10-03 14:16:00,36,B00474,Casual,,,
6,89335,9997,2011-10-09 02:30:00,60,2011-10-09 05:17:00,45,B00047,Casual,,,
7,124500,9992,2011-11-09 09:08:00,22,2011-11-09 11:55:00,40,B00387,Casual,,,
8,133967,9996,2011-11-19 13:48:00,4,2011-11-19 16:35:00,58,B00238,Casual,,,
9,147451,9996,2012-03-23 14:48:00,35,2012-03-23 17:35:00,33,B00550,Casual,,,


As you can see, the query returned 14 different trips, each with a duration of 9990 seconds or more. Something that stands out about this query is that all but one of the results has a sub_type of "Casual". Perhaps this is an indication that "Registered" users are more aware of the extra fees for long trips and maybe Hubway could do a better job of conveying their pricing structure to Casual users to help them avoid overage charges.

We can also combine multiple logical tests in our WHERE clause using AND or OR. If, for example, in our previous query we had only wanted to return the trips with a duration over 9990 seconds that also had a sub_type of Registered, we could use AND to specify both conditions. I also recommend using parentheses to separate each logical test. This isn’t strictly required for the code to function, but they can make your queries easier to understand as you increase the complexity. Let’s run that query now. We already know it should only return one result, so it should be easy to check that we’ve got it right:

In [27]:
query = '''
SELECT * 
FROM trips
WHERE (duration >= 9990) AND (sub_type = "Registered")
ORDER BY duration DESC;
'''

run_query(query)

,id,duration,start_date,start_station,end_date,end_station,bike_number,sub_type,zip_code,birth_date,gender
0,315737,9995,2012-07-03 18:28:00,12,2012-07-03 21:15:00,12,B00250,Registered,'02120,1964.0,Male


## How many trips were taken by ‘registered’ users?

In this instance, it doesn’t matter which column we choose to count because every column should have data for each row in our query, but sometimes a query might have missing (or “null”) values for some rows. If you’re not sure whether a column contains null values you can run your COUNT on the id column - the id column is never null, so you can be sure your count won’t have missed anything. You can also use COUNT(1) or COUNT(*) to count up every row in your query. It’s worth noting that sometimes you might actually want to run  COUNT on a column with null values, for example if you wanted to know how many rows in your database have missing values for a column.

Let’s take a look at a query to answer our question. We can use SELECT COUNT(*) to count up the total number of columns returned and WHERE sub_type = "Registered" to make sure we only count up the trips taken by Registered users.

In [14]:
query = '''
SELECT COUNT(*)
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,COUNT(*)
0,1105192


This query worked and has returned the answer to our question, but the column heading isn’t particularly descriptive. If you want to make your results more readable, you can use AS to give your output an alias (or nickname). If we wanted to re-run the previous query but give our column heading an alias of Total Trips by Registered Users, we could do the following:

In [15]:
query = '''
SELECT COUNT(*) AS "Total Trips by Registered Users"
FROM trips
WHERE sub_type = "Registered";
'''

run_query(query)

,Total Trips by Registered Users
0,1105192


### Aggregate Functions
COUNT is not the only trick SQL has in this domain. You can also use SUM, AVG, MIN and MAX to return the sum, average, minimum and maximum of a column respectively. These, along with COUNT, are known as aggregate functions. So to answer our third question, “What was the average trip duration?”, we can use the AVG function on the duration column (and, once again, use AS to give our output column a more descriptive name):



In [16]:
query = '''
SELECT AVG(duration) AS "Average Duration"
FROM trips;
'''

run_query(query)

,Average Duration
0,912.409682


In [30]:
query='''
SELECT MIN(duration) AS "MIN DURATION" FROM trips'''
print(run_query(query))

query='''
SELECT MAX(duration) AS "MAX DURATION" FROM trips'''
print(run_query(query))

   MIN DURATION
0             0
   MAX DURATION
0          9999


So it turns out that the average trip duration is 912 seconds, which is about 15 minutes. This makes some sense since we know that Hubway charges extra fees for trips over 30 minutes because the service is designed for riders to take short, one-way trips.

But what about our next question, do registered or casual users take longer trips? We already know one way to answer this question - we can run two SELECT AVG(duration) FROM trips queries with WHERE clauses that restrict one to "Registered" and one to "Casual" users. SQL also includes a way that we can answer this question in a single query, using the GROUP BY command.

In [17]:
query = '''
SELECT sub_type, AVG(duration) AS "Average Duration"
FROM trips
GROUP BY sub_type;
'''

run_query(query)

,sub_type,Average Duration
0,Casual,1519.643897
1,Registered,657.026067


Well that’s quite a difference. On average, registered users take trips that last around 11 minutes whereas casual users are spending almost 25 minutes per ride. Registered users are likely taking shorter, more frequent trips, possibly as part of their commute to work. Casual users, on the other hand, are spending around twice as long per trip. It’s possible that casual users tend to come from demographics (tourists, for example) that are more inclined to take longer trips make sure they get around and see all the sights.



###  which bike was used for the most trips

In [18]:
query = '''
SELECT bike_number as "Bike Number", COUNT(*) AS "Number of Trips"
FROM trips
GROUP BY bike_number
ORDER BY COUNT(*) DESC
LIMIT 1;
'''

run_query(query)

,Bike Number,Number of Trips
0,B00490,2120


## average duration of trips by registered members over the age of 30

In [19]:
query = '''
SELECT AVG(duration)
FROM trips
WHERE (2017 - birth_date) > 30;
'''

run_query(query)

,AVG(duration)
0,923.014685


In [31]:
### STATIONS TABLE

* id - A unique identifier for each station (corresponds to the start_station and end_station columns in the  trips table)
station - The station name
* municipality - The municipality that the station is in (Boston, Brookline, Cambridge or Somerville)
* lat - The latitude of the station
* lng - The longitude of the station

In [20]:
query = '''
SELECT *
FROM stations
LIMIT 5;
'''
run_query(query)

,id,station,municipality,lat,lng
0,3,Colleges of the Fenway,Boston,42.340021,-71.100812
1,4,Tremont St. at Berkeley St.,Boston,42.345392,-71.069616
2,5,Northeastern U / North Parking Lot,Boston,42.341814,-71.090179
3,6,Cambridge St. at Joy St.,Boston,42.361284999999995,-71.06514
4,7,Fan Pier,Boston,42.353412,-71.044624


## with which station is the most frequent starting point? Let’s work through it step by step:


* First we want to use SELECT to return the station column from the stations table and the COUNT of the number of rows.
* Next we specify the tables we want to JOIN and tell the database to connect them ON the start_station column in the trips table and the id column in the stations table.
* Then we get into the meat of our query - we GROUP BY the station column in the stations table so that our  COUNT will count up the number of trips for each station separately
* Finally we can ORDER BY our COUNT and LIMIT the output to a manageable number of results


In [21]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
INNER JOIN stations
ON trips.start_station = stations.id
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,South Station - 700 Atlantic Ave.,56123
1,Boston Public Library - 700 Boylston St.,41994
2,Charles Circle - Charles St. at Cambridge St.,35984
3,Beacon St / Mass Ave,35275
4,MIT at Mass Ave / Amherst St,33644


In [22]:
query = '''
SELECT stations.station AS "Station", COUNT(*) AS "Count"
FROM trips 
INNER JOIN stations
ON trips.start_station = stations.id
WHERE trips.start_station = trips.end_station
GROUP BY stations.station
ORDER BY COUNT(*) DESC
LIMIT 5;
'''

run_query(query)

,Station,Count
0,The Esplanade - Beacon St. at Arlington St.,3064
1,Charles Circle - Charles St. at Cambridge St.,2739
2,Boston Public Library - 700 Boylston St.,2548
3,Boylston St. at Arlington St.,2163
4,Beacon St / Mass Ave,2144


In [23]:
query = '''
SELECT COUNT(trips.id) AS "Count"
FROM trips 
INNER JOIN stations AS start
ON trips.start_station = start.id
INNER JOIN stations AS end
ON trips.end_station = end.id
WHERE start.municipality <> end.municipality;
'''

run_query(query)

,Count
0,309748
